In [1]:
def read_python_file(file_path):
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except FileNotFoundError:
        return "Error: File not found."

In [2]:
from langchain_core.tools import tool
from pydantic.v1 import BaseModel, Field
from typing import Optional

class codepath(BaseModel):
  path: str = Field(description="code path to execute")

@tool(args_schema = codepath)
def execute_query(path: str) -> str:
  """Returns the result of code path execution"""
  return read_python_file(path)

In [3]:
# useful imports
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
import ollama


# defining agent state
class AgentState(TypedDict):
   messages: Annotated[list[AnyMessage], operator.add]

In [4]:
from semantic_router.utils.function_call import FunctionSchema

# create the function calling schema for ollama
execute_query_schema = FunctionSchema(read_python_file).to_ollama()
# execute_query_schema["function"]["parameters"]["properties"]["description"] = None
execute_query_schema["function"]["parameters"]["properties"]["description"] = None
execute_query_schema

/Users/akhilred/Library/Python/3.13/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'type': 'function',
 'function': {'name': 'read_python_file',
  'description': 'None',
  'parameters': {'type': 'object',
   'properties': {'file_path': {'description': None, 'type': 'object'},
    'description': None},
   'required': []}}}

In [5]:
class codeAgent:
  # initialising the object
  def __init__(self, model, tools, system_prompt = ""):
    self.system_prompt = system_prompt

    # initialising graph with a state 
    graph = StateGraph(AgentState)

    # adding nodes 
    graph.add_node("llm", self.call_llm)
    graph.add_node("function", self.execute_function)
    graph.add_conditional_edges(
      "llm",
      self.exists_function_calling,
      {True: "function", False: END}
    )
    graph.add_edge("function", "llm")

    # setting starting point
    graph.set_entry_point("llm")

    self.graph = graph.compile()
    self.tools = {t.name: t for t in tools}
    self.model = model.bind_tools(tools)

  def call_llm(self, state: AgentState):
    messages = state['messages']
    # adding system prompt if it's defined
    if self.system_prompt:
        messages = [SystemMessage(content=self.system_prompt)] + messages

    # calling LLM
    message = self.model.invoke(messages)

    return {'messages': [message]}
  
  def execute_function(self, state: AgentState):
    tool_calls = state['messages'][-1].tool_calls

    results = []
    for tool in tool_calls:
      # checking whether tool name is correct
      if not tool['name'] in self.tools:
        # returning error to the agent 
        result = "Error: There's no such tool, please, try again" 
      else:
        # getting result from the tool
        result = self.tools[tool['name']].invoke(tool['args'])

      results.append(
        ToolMessage(
          tool_call_id=tool['id'], 
          name=tool['name'], 
          content=str(result)
        )
    )
    return {'messages': results}
  
  def exists_function_calling(self, state: AgentState):
    result = state['messages'][-1]
    return len(result.tool_calls) > 0


from langchain_ollama import ChatOllama
model = ChatOllama(model="llama3.2:1b")

# system prompt
prompt = '''You are a senior expert in reviewing python code. 
So, you can help the team to review the code and provide feedback. 
You are very accurate and take into account all the nuances in code.
Your goal is to provide the detailed documentation for any security issues in the code that will help users.'''

doc_agent = codeAgent(model, [execute_query], system_prompt=prompt)

messages = [HumanMessage(content="Does the code have any security issues the path is '/Users/akhilred/Desktop/Billing Pyton Script/Billing_Usage_Extraction.py'?")]
result = doc_agent.graph.invoke({"messages": messages})

2025-03-20 16:05:49 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-20 16:06:09 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


In [6]:
result

{'messages': [HumanMessage(content="Does the code have any security issues the path is '/Users/akhilred/Desktop/Billing Pyton Script/Billing_Usage_Extraction.py'?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-03-20T22:05:49.616083Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1380800750, 'load_duration': 773909917, 'prompt_eval_count': 247, 'prompt_eval_duration': 256885833, 'eval_count': 34, 'eval_duration': 347847459, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-e4a30c9c-1351-4c0f-826f-fd9268c50d2d-0', tool_calls=[{'name': 'execute_query', 'args': {'path': '/Users/akhilred/Desktop/Billing Pyton Script/Billing_Usage_Extraction.py'}, 'id': '9027526a-ba1c-4ed6-af66-4b4440a5631f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 247, 'output_tokens': 34, 'total_tokens': 281}),
  ToolMessage(content='# This Script is cr

In [7]:
print(result['messages'][3].content)

The provided code appears to be a Python script that uses the Azure Data Factory (ADF) to connect to various data sources and perform data transformations. Here are some potential issues and improvements:

1. **Authentication**: The authentication mechanism is not properly implemented. The `get_data_for_usage_commercial`, `get_data_for_usage_app_auth`, and `get_data_for_billing` functions do not handle authentication correctly.

2. **Exception handling**: The code does not handle exceptions well. For example, if the data source connection fails or there are errors during data transformations, an exception is caught but not handled properly.

3. **Error logging**: There is no error logging mechanism in place to track any errors that may occur during data processing.

4. **Input validation**: The code assumes that the input parameters (`sdt`, `edt`) will always be valid and do not validate them.

5. **Type hints**: The code does not use type hints, which can make it harder for others to 